In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style("whitegrid")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 200)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Load the dataset
df = pd.read_excel('_data/_matches_list_v2.xlsx')

In [3]:
##### SANITY CHECKS REGARDING THE BETTING ODDS #####

# Betting companies
companies = ['B365', 'EX', 'LB', 'PS', 'SJ', 'UB', 'Avg', 'Max']

# Columns for each player for each company
cols_p1 = [f'p1_{company}' for company in companies]
cols_p2 = [f'p2_{company}' for company in companies]

# Convert cols_p1 and cols_p2 to numeric
for col in cols_p1 + cols_p2:
    df[col] = pd.to_numeric(df[col], errors='coerce')

display(df[(df[cols_p1] < 1).any(axis=1) | (df[cols_p2] < 1).any(axis=1)])

# Update cols_p1 and cols_p2 to =1 if <1
for col in cols_p1 + cols_p2:
    df.loc[df[col] < 1, col] = 1

display(df[(df[cols_p1] < 1).any(axis=1) | (df[cols_p2] < 1).any(axis=1)])

,match_id,match_date,year,tourney_name,tourney_level,court,surface,draw_size,round,best_of,minutes,total_points,total_games,score,p1_won,p1_points,p2_points,p1_games,p2_games,p1_sets,p2_sets,p1_id,p2_id,p1_name,p2_name,p1_rank,p2_rank,p1_rankpt,p2_rankpt,p1_hand,p2_hand,p1_ht,p2_ht,p1_ioc,p2_ioc,p1_age,p2_age,p1_seed,p2_seed,p1_entry,p2_entry,p1_ace,p2_ace,p1_df,p2_df,p1_svpt,p2_svpt,p1_1stIn,p2_1stIn,p1_1stWon,p2_1stWon,p1_2ndWon,p2_2ndWon,p1_SvGms,p2_SvGms,p1_bpSaved,p2_bpSaved,p1_bpFaced,p2_bpFaced,p1_B365,p2_B365,p1_EX,p2_EX,p1_LB,p2_LB,p1_PS,p2_PS,p1_SJ,p2_SJ,p1_UB,p2_UB,p1_Max,p2_Max,p1_Avg,p2_Avg
28088,2020-580-0164,2020-01-23,2020,Australian Open,G,Outdoor,Hard,128,R64,5,150.00,199.00,29,6-3 7-6(4) 6-1,1,116.00,83.00,19,10,3,0,104745,105643,Rafael Nadal,Federico Delbonis,1.00,76.00,10235.00,711.00,L,L,185.00,190.00,ESP,ARG,33.60,29.20,1.00,NaN,NaN,NaN,8.00,2.00,1.00,8.00,89.00,110.00,61.00,69.00,52.00,45.00,17.00,18.00,14.00,14.00,0.00,17.00,0.00,20.00,29.00,0.97,NaN,NaN,NaN,NaN,1.01,29.89,NaN,NaN,NaN,NaN,1.02,35.00,1.01,21.52
28800,2020-520-1164,2020-09-28,2020,Roland Garros,G,Outdoor,Clay,128,R128,5,125.00,148.00,28,6-4 6-4 6-2,1,88.00,60.00,18,10,3,0,104745,106078,Rafael Nadal,Egor Gerasimov,2.00,83.00,9850.00,750.00,L,R,185.00,196.00,ESP,BLR,34.30,27.80,2.00,NaN,NaN,NaN,3.00,8.00,2.00,1.00,71.00,77.00,45.00,49.00,37.00,34.00,17.00,9.00,14.00,14.00,1.00,0.00,2.00,5.00,0.97,34.00,NaN,NaN,NaN,NaN,1.01,30.08,NaN,NaN,NaN,NaN,1.02,35.00,1.01,22.68
28836,2020-520-1232,2020-09-30,2020,Roland Garros,G,Outdoor,Clay,128,R64,5,100.00,132.00,22,6-1 6-0 6-3,1,85.00,47.00,18,4,3,0,104745,111456,Rafael Nadal,Mackenzie Mcdonald,2.00,236.00,9850.00,217.00,L,R,185.00,178.00,ESP,USA,34.30,25.40,2.00,NaN,NaN,NaN,1.00,1.00,4.00,2.00,72.00,60.00,49.00,40.00,37.00,18.00,14.00,8.00,11.00,11.00,0.00,1.00,0.00,8.00,1.01,26.00,NaN,NaN,NaN,NaN,0.97,36.86,NaN,NaN,NaN,NaN,1.02,36.86,1.01,22.51
28837,2020-520-1201,2020-10-01,2020,Roland Garros,G,Outdoor,Clay,128,R64,5,83.00,119.00,23,6-1 6-2 6-2,0,43.00,76.00,5,18,0,3,105575,104925,Ricardas Berankis,Novak Djokovic,66.00,1.00,854.00,11260.00,R,R,175.00,188.00,LTU,SRB,30.20,33.30,NaN,1.00,NaN,NaN,0.00,10.00,1.00,1.00,53.00,66.00,40.00,38.00,20.00,36.00,6.00,13.00,11.00,12.00,1.00,2.00,7.00,2.00,0.97,34.00,NaN,NaN,NaN,NaN,28.00,1.01,NaN,NaN,NaN,NaN,41.00,1.02,21.54,1.01
29388,2021-580-0164,2021-02-10,2021,Australian Open,G,Outdoor,Hard,128,R64,5,210.00,268.00,44,6-3 6-7(3) 7-6(2) 6-3,0,123.00,145.00,19,25,1,3,126207,104925,Frances Tiafoe,Novak Djokovic,64.00,1.00,1005.00,12030.00,R,R,188.00,188.00,USA,SRB,23.00,33.70,NaN,1.00,NaN,NaN,23.00,26.00,8.00,5.00,148.00,120.00,78.00,74.00,58.00,62.00,34.00,27.00,21.00,21.00,9.00,1.00,14.00,3.00,0.97,29.00,NaN,NaN,NaN,NaN,24.50,1.02,NaN,NaN,NaN,NaN,29.00,1.02,19.55,1.01
30309,2021-520-0100,2021-06-01,2021,Roland Garros,G,Outdoor,Clay,128,R128,5,118.00,167.00,26,6-2 6-4 6-2,0,68.00,99.00,8,18,0,3,105815,104925,Tennys Sandgren,Novak Djokovic,66.00,1.00,1033.00,11313.00,R,R,188.00,188.00,USA,SRB,29.80,34.00,NaN,1.00,NaN,NaN,5.00,4.00,1.00,2.00,94.00,73.00,56.00,49.00,30.00,42.00,20.00,13.00,13.00,13.00,6.00,6.00,11.00,6.00,0.97,29.00,NaN,NaN,NaN,NaN,31.38,1.01,NaN,NaN,NaN,NaN,35.00,1.02,23.01,1.01
30368,2021-520-0196,2021-06-05,2021,Roland Garros,G,Outdoor,Clay,128,R32,5,92.00,142.00,24,6-1 6-4 6-1,0,55.00,87.00,6,18,0,3,105575,104925,Ricardas Berankis,Novak Djokovic,93.00,1.00,839.00,11313.00,R,R,175.00,188.00,LTU,SRB,30.90,34.00,NaN,1.00,NaN,NaN,1.00,5.00,3.00,3.00,75.00,67.00,53.00,42.00,31.00,37.00,7.00,13.00,12.00,12.00,3.00,0.00,9.00,0.00,0.97,29.00,NaN,NaN,NaN,NaN,32.24,1.01,NaN,NaN,NaN,NaN,34.00,1.01,21.85,1.01
32820,2022-520-0187,2022-05-26,2022,Roland Garros,G,Outdoor,Clay,128,R64,5,NaN,328.00,48,6-3 7-6(8) 6-7(3) 7-6(7),1,167.00,161.00,26,22,3,1,126774,144645,Stefanos Tsitsipas,Zdenek Kolar,4.00,134.00,5965.00,468.00,R,R,193.00,185.00,GRE,CZE,23.70,25.60,4.00,NaN,NaN,NaN,25.00,9.00,5.00,7.00,163.00,165.00,96.00,117.00,70.00,82.00,38.00,24.00,23.00,22

,match_id,match_date,year,tourney_name,tourney_level,court,surface,draw_size,round,best_of,minutes,total_points,total_games,score,p1_won,p1_points,p2_points,p1_games,p2_games,p1_sets,p2_sets,p1_id,p2_id,p1_name,p2_name,p1_rank,p2_rank,p1_rankpt,p2_rankpt,p1_hand,p2_hand,p1_ht,p2_ht,p1_ioc,p2_ioc,p1_age,p2_age,p1_seed,p2_seed,p1_entry,p2_entry,p1_ace,p2_ace,p1_df,p2_df,p1_svpt,p2_svpt,p1_1stIn,p2_1stIn,p1_1stWon,p2_1stWon,p1_2ndWon,p2_2ndWon,p1_SvGms,p2_SvGms,p1_bpSaved,p2_bpSaved,p1_bpFaced,p2_bpFaced,p1_B365,p2_B365,p1_EX,p2_EX,p1_LB,p2_LB,p1_PS,p2_PS,p1_SJ,p2_SJ,p1_UB,p2_UB,p1_Max,p2_Max,p1_Avg,p2_Avg


In [4]:
##### REMOVE DEVIATED BETTING ODDS USING COEFFICIENT OF VARIATION TEST #####

# Define a threshold for CV
threshold_cv = 0.4

# Betting companies
companies = ['B365', 'EX', 'LB', 'PS', 'SJ', 'UB', 'Avg', 'Max']

# Columns for each player for each company
cols_p1 = [f'p1_{company}' for company in companies]
cols_p2 = [f'p2_{company}' for company in companies]

# Convert betting columns to numeric
for col in cols_p1 + cols_p2:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# DataFrame to store updated fields information
updated_fields_df = pd.DataFrame(columns=['match_id', 'updated_field', 'original_value', 'cv_odds', 'p1_rank', 'p2_rank', 'p1_won'] + cols_p1 + cols_p2)

# Function to append updated row
def append_updated_row(row, outlier_col, original_value, cv_odds):
    updated_row = {
        'match_id': row['match_id'], 
        'updated_field': outlier_col, 
        'original_value': original_value,
        'cv_odds': cv_odds,
        'p1_rank': row['p1_rank'], 
        'p2_rank': row['p2_rank'], 
        'p1_won': row['p1_won']
    }
    updated_row.update(row[cols_p1 + cols_p2].to_dict())
    return updated_row

# Iterate through the rows
for idx, row in df.iterrows():
    for cols in [cols_p1, cols_p2]:
        while True:
            # Calculate mean and standard deviation
            mean_odds = row[cols].mean()
            std_odds = row[cols].std()
            cv_odds = std_odds / mean_odds

            # If CV is greater than the threshold
            if cv_odds > threshold_cv:
                # Calculate the difference between each value and the mean
                differences = row[cols].apply(lambda x: abs(x - mean_odds))
                # Find the specific column causing the high CV
                outlier_col = differences.idxmax()
                # Save the original value
                original_value = row[outlier_col]
                # Update the specific cell with NaN
                df.at[idx, outlier_col] = np.nan
                # Update the row for further iteration
                row[outlier_col] = np.nan

                # Append to the DataFrame
                updated_fields_df = updated_fields_df.append(append_updated_row(row, outlier_col, original_value, cv_odds), ignore_index=True)
            else:
                break

# Sort the DataFrame by 'cv_odds' in descending order
updated_fields_df = updated_fields_df.sort_values(by='cv_odds', ascending=False)

# Filter out 'updated_field' == 'p1_Avg', 'p2_Avg', 'p1_Max', 'p2_Max' and print "Updated cells:" + total number of rows
print(f"Number of updated betting odds (only company odds): {len(updated_fields_df[~updated_fields_df['updated_field'].isin(['p1_Avg', 'p2_Avg', 'p1_Max', 'p2_Max'])])}")

# Display the DataFrame with updated fields information
updated_fields_df[~updated_fields_df['updated_field'].isin(['p1_Avg', 'p2_Avg', 'p1_Max', 'p2_Max'])]


Number of updated betting odds (only company odds): 209


,match_id,updated_field,original_value,cv_odds,p1_rank,p2_rank,p1_won,p1_B365,p1_EX,p1_LB,p1_PS,p1_SJ,p1_UB,p1_Avg,p1_Max,p2_B365,p2_EX,p2_LB,p2_PS,p2_SJ,p2_UB,p2_Avg,p2_Max
428,2020-520-1201,p2_B365,34.00,1.78,66.00,1.00,0,NaN,NaN,NaN,28.00,NaN,NaN,21.54,41.00,NaN,NaN,NaN,1.01,NaN,NaN,1.01,1.02
439,2021-520-0196,p2_B365,29.00,1.75,93.00,1.00,0,NaN,NaN,NaN,32.24,NaN,NaN,21.85,34.00,NaN,NaN,NaN,1.01,NaN,NaN,1.01,1.01
423,2020-580-0164,p1_B365,29.00,1.75,1.00,76.00,1,NaN,NaN,NaN,1.01,NaN,NaN,1.01,1.02,1.00,NaN,NaN,29.89,NaN,NaN,21.52,35.00
436,2021-520-0100,p2_B365,29.00,1.75,66.00,1.00,0,NaN,NaN,NaN,31.38,NaN,NaN,23.01,35.00,NaN,NaN,NaN,1.01,NaN,NaN,1.01,1.02
431,2021-580-0164,p2_B365,29.00,1.75,64.00,1.00,0,NaN,NaN,NaN,24.50,NaN,NaN,19.55,29.00,NaN,NaN,NaN,1.02,NaN,NaN,1.01,1.02
432,2021-0451-0284,p1_B365,51.00,1.55,NaN,45.00,0,NaN,NaN,NaN,3.26,NaN,NaN,3.40,3.75,1.00,NaN,NaN,1.38,NaN,NaN,1.31,1.42
301,2013-404-0089,p1_EX,8.00,1.24,1.00,8.00,1,1.08,NaN,1.11,1.11,1.13,NaN,1.10,1.14,8.00,6.71,6.50,8.50,6.50,NaN,6.88,8.70
14,2009-520-0038,p1_SJ,9.00,1.03,41.00,65.00,0,1.66,1.70,1.80,NaN,NaN,1.70,NaN,NaN,2.10,2.16,1.91,NaN,1.04,2.10,NaN,NaN
37,2010-1536-0040,p2_PS,86.00,0.93,3.00,62.00,1,NaN,1.01,1.01,1.01,1.01,NaN,1.01,1.02,21.00,15.00,21.00,NaN,15.00,NaN,14.89,21.00
6,2009-451-0030,p1_SJ,8.00,0.91,2.00,4.00,0,1.72,1.80,1.83,NaN,NaN,1.86,NaN,NaN,2.00,2.00,1.83,NaN,1.07,1.90,NaN,NaN


In [5]:
##### FILL MISSING BETTING ODDS WITH MEAN AND MAX OF THE OTHER ODDS #####

# Betting companies
companies = ['B365', 'EX', 'LB', 'PS', 'SJ', 'UB']

# Columns for each player for each company
cols_p1 = [f'p1_{company}' for company in companies]
cols_p2 = [f'p2_{company}' for company in companies]

# Calculate mean and max only for rows where not all odds of companies are missing
mask_p1 = df[cols_p1].notna().any(axis=1)
mask_p2 = df[cols_p2].notna().any(axis=1)

# Fill 'p1_Avg' and 'p2_Avg' with the mean of the betting odds where applicable
df.loc[mask_p1, 'p1_Avg'] = df.loc[mask_p1, cols_p1].mean(axis=1, skipna=True)
df.loc[mask_p2, 'p2_Avg'] = df.loc[mask_p2, cols_p2].mean(axis=1, skipna=True)

# Fill 'p1_Max' and 'p2_Max' with the maximum of the betting odds where applicable
df.loc[mask_p1, 'p1_Max'] = df.loc[mask_p1, cols_p1].max(axis=1, skipna=True)
df.loc[mask_p2, 'p2_Max'] = df.loc[mask_p2, cols_p2].max(axis=1, skipna=True)

# Print missing or null values in 'p1_Avg', 'p2_Avg', 'p1_Max', 'p2_Max' columns
print(df[['p1_Avg', 'p2_Avg', 'p1_Max', 'p2_Max']].isnull().sum())

# Filter rows where any of the specified columns have missing or null values
missing_df = df[df[['p1_Avg', 'p2_Avg', 'p1_Max', 'p2_Max']].isnull().any(axis=1)]
missing_df[['match_id', 'match_date', 'tourney_name', 'tourney_level', 'round', 'p1_name', 'p2_name', 'p1_won', 'score', 'p1_rank', 'p2_rank', 'p1_Avg', 'p2_Avg', 'p1_Max', 'p2_Max']]


p1_Avg    51
p2_Avg    51
p1_Max    51
p2_Max    51
dtype: int64


,match_id,match_date,tourney_name,tourney_level,round,p1_name,p2_name,p1_won,score,p1_rank,p2_rank,p1_Avg,p2_Avg,p1_Max,p2_Max
392,2009-424-0014,2009-02-11,San Jose,A,R32,John Isner,Ramon Delgado,1,7-6(9) 6-4,139.00,277.00,NaN,NaN,NaN,NaN
828,2009-717-0009,2009-04-07,Houston,A,R32,Amer Delic,Paul Capdeville,0,6-7(5) 7-6(5) 6-1,148.00,87.00,NaN,NaN,NaN,NaN
1061,2009-5053-0008,2009-05-05,Belgrade,A,R32,Dominik Hrbaty,Kristof Vliegen,0,2-6 6-2 6-2,160.00,105.00,NaN,NaN,NaN,NaN
1062,2009-5053-0009,2009-05-05,Belgrade,A,R32,Arsenije Zlatanovic,Lukasz Kubot,0,6-3 7-5,1421.00,179.00,NaN,NaN,NaN,NaN
1063,2009-5053-0010,2009-05-05,Belgrade,A,R32,Santiago Ventura,Arnaud Clement,0,6-1 6-4,117.00,57.00,NaN,NaN,NaN,NaN
1067,2009-5053-0011,2009-05-06,Belgrade,A,R32,Marcos Baghdatis,Flavio Cipolla,0,1-6 6-3 6-1,87.00,122.00,NaN,NaN,NaN,NaN
1125,2009-1536-0024,2009-05-11,Madrid Masters,M,R64,Robin Soderling,Ivan Navarro,1,6-4 6-2,23.00,84.00,NaN,NaN,NaN,NaN
1173,2009-306-0007,2009-05-19,Kitzbuhel,A,R32,Mario Ancic,Ruben Ramirez Hidalgo,0,6-4 6-3,38.00,131.00,NaN,NaN,NaN,NaN
1227,2009-520-0059,2009-05-25,Roland Garros,G,R128,Jeremy Chardy,Thiago Alves,1,6-2 7-6(10) 6-3,39.00,117.00,NaN,NaN,NaN,NaN
1360,2009-311-0032,2009-06-10,Queen's Club,A,R32,Xavier Malisse,Rik De Voest,1,7-6(6) 7-6(8),172.00,206.00,NaN,NaN,NaN,NaN


In [6]:
 # Drop company odds columns
df = df.drop(columns=cols_p1 + cols_p2)

In [7]:
# Export dataset to excel
df.to_excel('_data/_matches_list_v3.xlsx', index=False)